# Pipeline di Addestramento StackGAN: Fase I e Fase II

Questo notebook gestisce l'intero processo di addestramento per il modello StackGAN, suddiviso in due fasi principali. Ogni cella è progettata per essere eseguita in sequenza.

1.  **Setup dell'Ambiente**: Installa le dipendenze necessarie dal file `requirements.txt`.
2.  **Addestramento Fase I**: Avvia lo script per addestrare la GAN a generare immagini a bassa risoluzione (64x64).
3.  **Visualizzazione Loss Fase I**: Mostra un grafico dell'andamento delle loss del generatore e del discriminatore durante la Fase I.
4.  **Addestramento Fase II**: Avvia lo script per addestrare la seconda GAN, che prende le immagini a bassa risoluzione e genera immagini ad alta risoluzione (256x256).
5.  **Visualizzazione Loss Fase II**: Mostra un grafico dell'andamento delle loss per la Fase II, incluse le componenti avversaria e L1.

In [ ]:
import os
import sys
import subprocess
import shutil

# --- 0. CLONAZIONE DEL REPOSITORY (SE NECESSARIO) ---
GIT_REPO_URL = "https://github.com/Biobay/DeepLearning/"
BRANCH_NAME = "RIS-1-CORRETTO"

# Controlla se il progetto è già stato clonato verificando l'esistenza di una cartella chiave
if not os.path.exists("scripts"):
    print(f"Clonazione del branch '{BRANCH_NAME}' da '{GIT_REPO_URL}'...")
    TEMP_DIR = "temp_clone"
    
    try:
        # Clona il repository in una cartella temporanea
        subprocess.run(['git', 'clone', '--branch', BRANCH_NAME, GIT_REPO_URL, TEMP_DIR], check=True)
        
        # Copia il contenuto dalla cartella temporanea a quella corrente, ignorando la .git
        print("Copia dei file di progetto nella directory corrente...")
        shutil.copytree(TEMP_DIR, ".", dirs_exist_ok=True, ignore=shutil.ignore_patterns('.git'))

        # Pulisce la cartella temporanea
        shutil.rmtree(TEMP_DIR)
        print("Clonazione completata.")
    except Exception as e:
        print(f"Errore durante la clonazione: {e}")
        # Se la clonazione fallisce, è inutile continuare
        sys.exit(f"Impossibile clonare il repository. Lo script verrà interrotto.")
else:
    print("La cartella del progetto sembra esistere già. Salto la clonazione.")


# --- 1. SETUP DELL'AMBIENTE ---

# Aggiunge la directory del progetto al path di sistema per garantire che gli import funzionino
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
    print("Path di sistema aggiornato.")
else:
    print("Path di sistema già configurato.")

# Installa le dipendenze
print("\nInstallazione delle dipendenze da requirements.txt...")
try:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], check=True, capture_output=True, text=True)
    print("Dipendenze installate con successo.")
except subprocess.CalledProcessError as e:
    print("Errore durante l'installazione delle dipendenze:")
    print(e.stderr)


---
### Fase I: Addestramento del Generatore e Discriminatore a 64x64

Questa cella avvia lo script `train.py` per addestrare la prima fase del modello.

In [ ]:
# --- 2. ADDESTRAMENTO FASE I ---
print("--- Avvio Addestramento Fase I ---")
try:
    from scripts.train import train
    import src.config as config
    
    # Pulisce l'output della cella prima di iniziare per maggiore leggibilità
    from IPython.display import clear_output
    clear_output(wait=True)
    
    print("Moduli importati. Inizio dell'addestramento S1...")
    train(config)
    print("--- Addestramento Fase I completato. ---")
    
except Exception as e:
    print(f"Si è verificato un errore durante l'addestramento della Fase I: {e}")


---
### Visualizzazione Risultati Fase I

Questa cella legge il file di log `loss_log.csv` e visualizza l'andamento delle loss.

In [ ]:
# --- 3. VISUALIZZAZIONE LOSS FASE I ---
import pandas as pd
import matplotlib.pyplot as plt
import os
from src import config # Importa la configurazione per usare i percorsi assoluti

LOG_FILE_PATH = os.path.join(config.LOG_DIR, "loss_log.csv")

if os.path.exists(LOG_FILE_PATH):
    print(f"Trovato file di log in: {LOG_FILE_PATH}")
    
    df_loss_s1 = pd.read_csv(LOG_FILE_PATH)
    df_loss_epoch_s1 = df_loss_s1.groupby('epoch').mean()
    
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(15, 7))

    ax.plot(df_loss_epoch_s1.index, df_loss_epoch_s1['loss_d'], 'o-', label='Avg Discriminator Loss (S1)', color='red')
    ax.plot(df_loss_epoch_s1.index, df_loss_epoch_s1['loss_g'], 'o-', label='Avg Generator Loss (S1)', color='blue')
    ax.plot(df_loss_epoch_s1.index, df_loss_epoch_s1['loss_g_adv'], '--', label='Avg Generator Loss (Adversarial)', color='cyan', alpha=0.7)
    ax.plot(df_loss_epoch_s1.index, df_loss_epoch_s1['loss_g_l1'], '--', label='Avg Generator Loss (L1)', color='lightblue', alpha=0.7)

    ax.set_title('Andamento Medio delle Loss per Epoca (Stage-I)', fontsize=16)
    ax.set_xlabel('Epoca', fontsize=12)
    ax.set_ylabel('Valore Medio Loss', fontsize=12)
    ax.legend()
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print(f"ERRORE: File di log non trovato in '{LOG_FILE_PATH}'. Esegui prima la cella di training della Fase I.")


---
### Fase II: Addestramento del Generatore e Discriminatore a 256x256

Questa cella avvia lo script `train_s2.py` per addestrare la seconda fase del modello, che raffina le immagini della Fase I.

In [ ]:
# --- 4. ADDESTRAMENTO FASE II ---
print("--- Avvio Addestramento Fase II ---")
try:
    from scripts.train_s2 import train_stage2
    
    # Pulisce l'output della cella
    from IPython.display import clear_output
    clear_output(wait=True)
    
    print("Modulo importato. Inizio dell'addestramento S2...")
    train_stage2()
    print("--- Addestramento Fase II completato. ---")

except Exception as e:
    print(f"Si è verificato un errore durante l'addestramento della Fase II: {e}")


---
### Visualizzazione Risultati Fase II

Questa cella legge il file di log `loss_log_s2.csv` e visualizza l'andamento delle loss per la seconda fase.

In [ ]:
# --- 5. VISUALIZZAZIONE LOSS FASE II ---
import pandas as pd
import matplotlib.pyplot as plt
import os
from src import config

LOG_FILE_PATH_S2 = os.path.join(config.LOG_DIR, "loss_log_s2.csv")

if os.path.exists(LOG_FILE_PATH_S2):
    print(f"Trovato file di log in: {LOG_FILE_PATH_S2}")
    
    df_loss_s2 = pd.read_csv(LOG_FILE_PATH_S2)
    df_loss_epoch_s2 = df_loss_s2.groupby('epoch').mean()
    
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(15, 7))

    ax.plot(df_loss_epoch_s2.index, df_loss_epoch_s2['d_loss'], 'o-', label='Avg Discriminator Loss (S2)', color='darkorange')
    ax.plot(df_loss_epoch_s2.index, df_loss_epoch_s2['g_loss'], 'o-', label='Avg Total Generator Loss (S2)', color='darkgreen')
    
    # Plotta anche le componenti della loss del generatore se esistono
    if 'g_loss_adv' in df_loss_epoch_s2.columns:
        ax.plot(df_loss_epoch_s2.index, df_loss_epoch_s2['g_loss_adv'], '--', label='Avg Generator Loss (Adversarial)', color='limegreen', alpha=0.8)
    if 'g_loss_l1' in df_loss_epoch_s2.columns:
        ax.plot(df_loss_epoch_s2.index, df_loss_epoch_s2['g_loss_l1'], '--', label='Avg Generator Loss (L1)', color='mediumseagreen', alpha=0.8)

    ax.set_title('Andamento Medio delle Loss per Epoca (Stage-II)', fontsize=16)
    ax.set_xlabel('Epoca', fontsize=12)
    ax.set_ylabel('Valore Medio Loss', fontsize=12)
    ax.legend()
    ax.grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print(f"ERRORE: File di log non trovato in '{LOG_FILE_PATH_S2}'. Esegui prima la cella di training della Fase II.")


---
### Generazione di Immagini

Usa questa cella per generare una nuova immagine basata su una descrizione testuale, utilizzando i modelli addestrati.

In [ ]:
# --- 6. GENERAZIONE IMMAGINE FINALE ---
import torch
from scripts.generate_image import generate_image_from_text
from src import config
from PIL import Image

# --- PARAMETRI ---
TEXT_PROMPT = "un pokemon rosso con le ali"
OUTPUT_FILENAME = "pokemon_generato.png"

# Percorso completo per l'output
output_path = os.path.join(config.RESULTS_DIR, OUTPUT_FILENAME)

print(f"Richiesta generazione per il prompt: '{TEXT_PROMPT}'")

try:
    # Genera l'immagine
    final_image_tensor = generate_image_from_text(
        prompt=TEXT_PROMPT,
        cfg=config
    )
    
    # Converte il tensore in un'immagine PIL e la salva
    # Il tensore è normalizzato in [-1, 1], quindi lo riportiamo a [0, 1]
    final_image = (final_image_tensor.squeeze().permute(1, 2, 0) * 0.5 + 0.5).cpu().numpy()
    final_image = (final_image * 255).astype('uint8')
    img_pil = Image.fromarray(final_image)
    
    # Salva e mostra l'immagine
    img_pil.save(output_path)
    print(f"Immagine salvata con successo in: {output_path}")
    
    plt.imshow(img_pil)
    plt.title(f"Prompt: '{TEXT_PROMPT}'")
    plt.axis('off')
    plt.show()

except Exception as e:
    print(f"Si è verificato un errore durante la generazione dell'immagine: {e}")
